In [53]:
import cv2
import numpy as np
import pygame

# Initialize pygame mixer
pygame.mixer.init()

# Load the buzzer sound
buzzer_sound = pygame.mixer.Sound("cristiano-ronaldo-siuu-made-with-Voicemod-technology.mp3")  # Replace with the path to your sound file

cap = cv2.VideoCapture(0)

# Load the pre-trained face cascade classifier from OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

yawn_counter = 0
sound_played = False  # Flag to track whether the sound has been played

def calculate_mouth_aspect_ratio(mouth):
    x, y, w, h = cv2.boundingRect(mouth)
    aspect_ratio = w // h
    return aspect_ratio

while True:
    # Capture frame
    ret, frame = cap.read()

    # Convert the frame to RGB 
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Convert the frame to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces using the Haar Cascade classifier
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Enlarge the face circle
        scale_factor = 0.6
        radius = int(max(w, h) * scale_factor)
        center = (x + w // 2, y + h // 2)
        cv2.circle(frame, center, radius, (255, 0, 0), 2)

        # Define the region of interest for mouth detection 
        roi_face_rgb = rgb_frame[y + h//2 + h//4:y + h, x:x + w]

        # Define the color range for detecting the mouth in RGB
        lower_color = np.array([100, 50, 0], dtype=np.uint8)
        upper_color = np.array([205, 140, 140], dtype=np.uint8)

        # Create mask for the mouth region
        mouth_mask = cv2.inRange(roi_face_rgb, lower_color, upper_color)

        # Find contours in the mask
        contours, _ = cv2.findContours(mouth_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Find the contour with the largest area
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)

            # Check if the bounding box width and height are above a certain threshold
            x_mouth, y_mouth, w_mouth, h_mouth = cv2.boundingRect(largest_contour)
            if w_mouth > 20 and h_mouth > 20:
                # Draw a rectangle around the detected mouth
                cv2.rectangle(frame, (x + x_mouth, y + h//2 + h//4 + y_mouth), 
                              (x + x_mouth + w_mouth, y + h//2 + h//4 + y_mouth + h_mouth), (0, 255, 0), 1)

                # Check for yawn based on mouth aspect ratio
                mouth_aspect_ratio = calculate_mouth_aspect_ratio(largest_contour)
                if h_mouth > 40 and not sound_played:
                    yawn_counter += 1
                    cv2.putText(frame, f'Yawn Count: {yawn_counter}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0),
                                2, cv2.LINE_AA)
                    
                    if yawn_counter > 40:
                        pygame.mixer.Sound.play(buzzer_sound)
                        sound_played = True  # Set the flag to True after playing the sound
                else:
                    yawn_counter = 0
                    sound_played = False  # Reset the flag if no yawn is detected

    cv2.imshow('Yawn Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
pygame.mixer.quit()
